In [0]:
import requests
import json
import time
from datetime import datetime
from typing import Dict, Any, List

# Function to extract required fields
def extract_core_details(prop: Dict[str, Any]) -> Dict[str, Any]:
    try:
        return {
            "zpid": prop.get("zpid"),
            "address": prop.get("address"),
            "city": prop.get("city"),
            "state": prop.get("state"),
            "zipcode": prop.get("zipcode"),
            "latitude": prop.get("latitude"),
            "longitude": prop.get("longitude"),
            "price": prop.get("price"),
            "zestimate": prop.get("zestimate"),
            "bedrooms": prop.get("bedrooms"),
            "bathrooms": prop.get("bathrooms"),
            "livingAreaValue": prop.get("livingArea"),
            "yearBuilt": prop.get("yearBuilt"),
            "homeType": prop.get("homeType"),
            "description": prop.get("hdpData", {}).get("homeInfo", {}).get("homeDescription"),
            "imgSrc": prop.get("imgSrc"),
            "agent_name": prop.get("hdpData", {}).get("homeInfo", {}).get("brokerName"),
            "agent_phone": None,
            "datePosted": None,
            "DateSold": prop.get("hdpData", {}).get("homeInfo", {}).get("dateSold")
        }
    except Exception as e:
        print(f"Error extracting details: {e}")
        return None

# Main logic
try:
    url = "https://zillow-com1.p.rapidapi.com/propertyExtendedSearch"
    querystring = {"location": "Los Angeles, CA"}
    api_key = dbutils.secrets.get(scope="zillowsecrets", key="rapidapi_key")
    headers = {
        "x-rapidapi-key": api_key,
        "x-rapidapi-host": "zillow-com1.p.rapidapi.com"
    }

    response = requests.get(url, headers=headers, params=querystring)
    response.raise_for_status()

    core_details_data = []
    if response.status_code == 200:
        data = response.json()
        properties = data.get("props", [])

        for prop in properties:
            details = extract_core_details(prop)
            if details:
                core_details_data.append(details)
            time.sleep(6)

        if core_details_data:
            print(f" Extracted {len(core_details_data)} properties.")
        else:
            print(" No property data found.")
    else:
        print(f"Unexpected status code: {response.status_code}")
except Exception as e:
    dbutils.jobs.taskValues.set(key="error", value=str(e))
    print(f" An error occurred during API call: {e}")
    raise


In [0]:
try:
    # Widget inputs
    container_name = dbutils.widgets.get("container_name")
    storage_account_name = dbutils.widgets.get("storage_account_name")
    base_folder_name = dbutils.widgets.get("base_folder_name")
    main_folder_name = dbutils.widgets.get("main_folder_name")
    storage_account_key = dbutils.secrets.get(scope="zillowsecrets", key="storage_account_key")

    # Configure access
    spark.conf.set(
        f"fs.azure.account.key.{storage_account_name}.dfs.core.windows.net",
        storage_account_key
    )

    # Construct path
    folder_path = f"abfss://{container_name}@{storage_account_name}.dfs.core.windows.net/{base_folder_name}/{main_folder_name}"
    timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    file_name = f"{timestamp}.json"
    file_path = f"{folder_path}/{file_name}"

    # Convert to JSON
    json_data = json.dumps(core_details_data, indent=2)

    # Write to ADLS
    dbutils.fs.put(file_path, json_data, overwrite=True)
    print(f" JSON written to: {file_path}")

except Exception as e:
    dbutils.jobs.taskValues.set(key="error", value=str(e))
    print(f" An error occurred while saving to ADLS: {e}")
    raise
